# Evaluation of the 1D validation plate (exp101/JG406–408)

We have to make the predictions for this plate and compare to the experimental outcome

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

from src.util.db_utils import SynFermDatabaseConnection
from src.definitions import DATA_DIR

In [ ]:
con = SynFermDatabaseConnection()

In [ ]:
# gather the experimental data
res = con.con.execute("SELECT e.id, vl_id, plate_nr, well, e.initiator_long, e.monomer_long, e.terminator_long, b.SMILES AS 'initiator', b2.SMILES AS 'monomer', b3.SMILES AS 'terminator', v.reaction_smiles_atom_mapped, product_A_lcms_ratio, product_B_lcms_ratio, product_C_lcms_ratio FROM experiments e LEFT JOIN virtuallibrary v on e.vl_id = v.id LEFT JOIN building_blocks b on e.initiator_long = b.long LEFT JOIN building_blocks b2 ON e.monomer_long = b2.long LEFT JOIN building_blocks b3 ON e.terminator_long = b3.long WHERE exp_nr = 101 AND (valid NOT LIKE '%ERROR%' OR valid IS NULL);").fetchall()
result = pd.DataFrame(res, columns=["id", "vl_id", "plate_nr", "well", "initiator_long", "monomer_long", "terminator_long", "initiator", "monomer", "terminator", "reaction_smiles_atom_mapped", "product_A_lcms_ratio", "product_B_lcms_ratio", "product_C_lcms_ratio"])
result["binary_A"] = (result["product_A_lcms_ratio"] > 0).astype(int)
result["binary_B"] = (result["product_B_lcms_ratio"] > 0).astype(int)
result["binary_C"] = (result["product_C_lcms_ratio"] > 0).astype(int)
result.head()

In [ ]:
# check how many of the 960 reactions were valid
len(result)

In [ ]:
# load plate data used for inference (we only need the vl_id to match with the experimental results)
val_plate = pd.read_csv(DATA_DIR / "curated_data" / "validation_plates.csv")[["vl_id"]]
val_plate.head()

In [ ]:
# load the predictions
preds = pd.read_csv(DATA_DIR / "curated_data" / "validation_plates_pred_2024-04-18.csv")
# merge plate data with preds
preds = pd.concat([val_plate, preds], axis=1)
preds.head()

In [ ]:
# combine predictions and results
comb = result.merge(preds, on="vl_id", how="left")
comb.head()

In [ ]:
# evaluate for binary_A
print(f'Accuracy: {accuracy_score(comb["binary_A"], comb["pred_A"]):.2%}')
print(f'Precision: {precision_score(comb["binary_A"], comb["pred_A"]):.2%}')
print(f'Recall: {recall_score(comb["binary_A"], comb["pred_A"]):.2%}')
print(f'Confusion: {confusion_matrix(comb["binary_A"], comb["pred_A"])}')

In [ ]:
# evaluate for binary_B
print(f'Accuracy: {accuracy_score(comb["binary_B"], comb["pred_B"]):.2%}')
print(f'Precision: {precision_score(comb["binary_B"], comb["pred_B"]):.2%}')
print(f'Recall: {recall_score(comb["binary_B"], comb["pred_B"]):.2%}')
print(f'Confusion: {confusion_matrix(comb["binary_B"], comb["pred_B"])}')

In [ ]:
# evaluate for binary_C
print(f'Accuracy: {accuracy_score(comb["binary_C"], comb["pred_C"]):.2%}')
print(f'Precision: {precision_score(comb["binary_C"], comb["pred_C"]):.2%}')
print(f'Recall: {recall_score(comb["binary_C"], comb["pred_C"]):.2%}')
print(f'Confusion: {confusion_matrix(comb["binary_C"], comb["pred_C"])}')

In [ ]:
# does this cluster by initiator?
for i in comb["initiator_long"].drop_duplicates():
    print(i)
    select = comb.loc[comb["initiator_long"] == i]
    print(f'Accuracy: {accuracy_score(select["binary_A"], select["pred_A"]):.2%}')
    print(f'Confusion: {confusion_matrix(select["binary_A"], select["pred_A"])}')

n.b. (because I keep forgetting) sklearn confusion matrix structure is [[tn fp]  [fn tp]]

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [ ]:
# show the wells that had valid reactions in the plate layout
# n.b. we ignore the right half of the plate b/c all of that was invalid (oxalic acid transfer error)
arr = np.zeros((3, 16, 20), dtype=int)
for plate in comb["plate_nr"]:
    for well in comb.loc[comb["plate_nr"] == plate, "well"]:
        row = ord(well[0]) - 65
        col = int(well[1:]) - 3
        arr[plate-1, row, col] = 1
arr

In [ ]:
# show the wells that had valid reactions in the plate layout
# n.b. we ignore the right half of the plate b/c all of that was invalid (oxalic acid transfer error)
arr = np.zeros((3, 16, 20), dtype=int)
for plate in comb["plate_nr"]:
    for well in comb.loc[comb["plate_nr"] == plate, "well"]:
        row = ord(well[0]) - 65
        col = int(well[1:]) - 3
        arr[plate-1, row, col] = 1
arr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(3)
sns.heatmap(arr[0], ax=ax[0])
sns.heatmap(arr[1], ax=ax[1])
sns.heatmap(arr[2], ax=ax[2])

In [ ]:
palette = sns.color_palette(["#e42536", "#f0f0f0", "#5790fc"])


In [ ]:
# show where we got it right for product A
arr = np.zeros((3, 16, 20), dtype=int)
for plate in comb["plate_nr"]:
    for i, dfrow in comb.loc[comb["plate_nr"] == plate].iterrows():
        row = ord(dfrow["well"][0]) - 65
        col = int(dfrow["well"][1:]) - 3
        if dfrow["pred_A"] == dfrow["binary_A"]:
            arr[plate-1, row, col] = 1
        else:
            arr[plate-1, row, col] = -1
fig, ax = plt.subplots(3)
sns.heatmap(arr[0], ax=ax[0], center=0, cmap=palette, cbar=False, linewidths=0.1)
sns.heatmap(arr[1], ax=ax[1], center=0, cmap=palette, cbar=False, linewidths=0.1)
sns.heatmap(arr[2], ax=ax[2], center=0, cmap=palette, cbar=False, linewidths=0.1)
for a in ax:
    a.set_xticks([])
    a.set_yticks([])
fig.savefig("exp101_accuracyA.pdf", transparent=True)

#### Note
Note how almost all of the wrong predictions for A originate from the first two rows. Which building blocks are these?

In [ ]:
comb.loc[comb["well"].str.startswith("A") | comb["well"].str.startswith("B"), "terminator_long"].drop_duplicates()

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

In [ ]:
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(con.get_smiles(long="TerABT007")), Chem.MolFromSmiles(con.get_smiles(long="TerABT012"))]
)

In [ ]:
# average success rate for TerABT007
con.con.execute("SELECT AVG(product_A_lcms_ratio > 0) FROM experiments WHERE terminator_long = 'TerABT007' AND exp_nr BETWEEN 4 AND 29 AND (valid NOT LIKE 'ERROR%' OR valid IS NULL);").fetchone()

In [ ]:
# average success rate for TerABT012
con.con.execute("SELECT AVG(product_A_lcms_ratio > 0) FROM experiments WHERE terminator_long = 'TerABT012' AND exp_nr BETWEEN 4 AND 29 AND (valid NOT LIKE 'ERROR%' OR valid IS NULL);").fetchone()

The success rate for TerABT007 is indeed markedly lower than for most building blocks giving a hint to why this may have been predicted not to work. For TerABT012 however that is less true. Still the high structural similarity to TerABT007 may play a role.

In [ ]:
# show where we got it right for product B
arr = np.zeros((3, 16, 20), dtype=int)
for plate in comb["plate_nr"]:
    for i, dfrow in comb.loc[comb["plate_nr"] == plate].iterrows():
        row = ord(dfrow["well"][0]) - 65
        col = int(dfrow["well"][1:]) - 3
        if dfrow["pred_B"] == dfrow["binary_B"]:
            arr[plate-1, row, col] = 1
        else:
            arr[plate-1, row, col] = -1
fig, ax = plt.subplots(3)
sns.heatmap(arr[0], ax=ax[0], center=0, cmap=sns.color_palette("coolwarm_r", n_colors=3))
sns.heatmap(arr[1], ax=ax[1], center=0, cmap=sns.color_palette("coolwarm_r", n_colors=3))
sns.heatmap(arr[2], ax=ax[2], center=0, cmap=sns.color_palette("coolwarm_r", n_colors=3))

In [ ]:
# show where we got it right for product C
arr = np.zeros((3, 16, 20), dtype=int)
for plate in comb["plate_nr"]:
    for i, dfrow in comb.loc[comb["plate_nr"] == plate].iterrows():
        row = ord(dfrow["well"][0]) - 65
        col = int(dfrow["well"][1:]) - 3
        if dfrow["pred_C"] == dfrow["binary_C"]:
            arr[plate-1, row, col] = 1
        else:
            arr[plate-1, row, col] = -1
fig, ax = plt.subplots(3)
sns.heatmap(arr[0], ax=ax[0], center=0, cmap=sns.color_palette("coolwarm_r", n_colors=3))
sns.heatmap(arr[1], ax=ax[1], center=0, cmap=sns.color_palette("coolwarm_r", n_colors=3))
sns.heatmap(arr[2], ax=ax[2], center=0, cmap=sns.color_palette("coolwarm_r", n_colors=3))